# S&P 500 Stock Screener
In this notebook is the first project of the summer, where the main goal is to become more comforatble with using packages like yfinance, pandas, and matplotlib.

3 goals of this project are to:
* Write code to fetch P/E Ratio, Market Cap, and Company Name for one ticker
* Print the results cleanly
* Understand what each value means and how it's structured

Some sub goals for myself are making sure that this code is written cleanly, and taking care and pride in this work. I have been reading Steve Job's biography, and onething that he preached was putting care into all aspects of a project, even that parts that can't be seen. In this case that will be formatting and commenting my code, as well as trying to make the UI as userfriendly as possible.

## Fetching Historical Market Data of Apple
We are going to analyze apple stock as an example

In [40]:
# Packages needed to import
import yfinance as yf
import pandas as pd
import numpy as np

In [26]:
# Define the ticker symbol
ticker_symbol = "AAPL"

# Create a Ticker object
ticker = yf.Ticker(ticker_symbol)

# Fetch Historical market data
historical_data = ticker.history(period = "1y") # Pulls data for the last year
print("Historical Data:")
print(historical_data)

# Fetch basic financials
financials = ticker.financials
print("\nFinaancials:")
print(financials)

# Fetch stock actions like dividends and splits
actions = ticker.actions
print("\nStock Actions:")
print(actions)



Historical Data:
                                 Open        High         Low       Close  \
Date                                                                        
2024-06-25 00:00:00-04:00  208.176416  210.396046  207.638936  208.096802   
2024-06-26 00:00:00-04:00  210.515486  213.859846  209.659488  212.257339   
2024-06-27 00:00:00-04:00  213.690659  214.735774  211.361555  213.103409   
2024-06-28 00:00:00-04:00  214.765593  215.064199  209.321054  209.639557   
2024-07-01 00:00:00-04:00  211.102731  216.497500  210.933525  215.741043   
...                               ...         ...         ...         ...   
2025-06-17 00:00:00-04:00  197.199997  198.389999  195.210007  195.639999   
2025-06-18 00:00:00-04:00  195.940002  197.570007  195.070007  196.580002   
2025-06-20 00:00:00-04:00  198.240005  201.699997  196.860001  201.000000   
2025-06-23 00:00:00-04:00  201.630005  202.300003  198.960007  201.500000   
2025-06-24 00:00:00-04:00  202.589996  203.440002  200.1999

### This next block of code we are going to Fetch:
* P/E Ratio
* Market Cap
* Company Name

Some background on what P/E ratio is,
    P/E (Price-to-Earning) Ratio is a measure of a company's share price relative to it's earning. This means a high P/E could mean that the compan'ys stock is overvalued, or it's expected to grow.

    The forumla to calculate P/E Ratio is (Market Value per Share)/(Earning Per Share)

    I am going to find the stock price throught the ticker and
    EPS = (Net Income)/(Common Stock Outstanding)

In [ ]:
# This is just me looking at the data that I get when I use history for one day
print(ticker.history(period = "1d"))

                                 Open        High         Low       Close  \
Date                                                                        
2025-06-24 00:00:00-04:00  202.589996  203.440002  200.199997  200.300003   

                             Volume  Dividends  Stock Splits  
Date                                                          
2025-06-24 00:00:00-04:00  53697168        0.0           0.0  


In [ ]:
price = ticker.history(period="1d")["Close"].iloc[0] # Price
eps = ticker.info.get("trailingEps") # Earning Per Share

pe = price/eps # P/E Ratio
print(pe)

31.150855840086752


### Now let's try to loop through multiple stock tickers

First let's break this into smaller tasks:

1) Be able to pull stock info from the web
2) Store that info in a list
3) Iterate through that list and figure out the P/E ration of each ticker

In [ ]:
# Lets pull 10 stocks to start from the web
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
tables = pd.read_html(url)

# Looks at the first table
sp5000_table = tables[0]

# Now we need the tickers
tickers = sp5000_table["Symbol"].head(10).tolist()
print(tickers)

# Awesome, now we've got a list of tickers

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']


Now that we've got a list of tickers let's loop through them and pull their P/E
We are going to store their info in a DF

In [51]:
# We Need to create a dataframe to store the tickers and their general information
# Looking for current price, P/E, and Market Cap
df = pd.DataFrame(columns=["Current Price", "P/E", "Market Cap"], index = tickers)
#  Loop through the tickers list
for ticker in tickers:
    try :
        # Grabs all the required information necessary to fill the table
        stock = yf.Ticker(ticker)
        price = stock.history(period="1d")["Close"].iloc[0]
        eps = stock.info.get("trailingEps")
        pe = price/eps
        market_cap = stock.info.get("marketCap")

        # Ads the info into the DataFrame
        df.loc[ticker] = [price, eps, market_cap]
    except:
        pass

print(df)


     Current Price    P/E    Market Cap
MMM     150.759995   8.04   81136164864
AOS      64.669998   3.58    9242765312
ABT     133.669998    7.7  239054094336
ABBV    186.789993   2.33  329945841664
ACN     296.079987  12.58  184414306304
ADBE    384.950012  15.63  165173968896
AMD     143.679993   1.36  232962752512
AES          10.81   1.84    7695887872
AFL     103.910004   6.43   56178421760
A       119.419998   4.06   33923041280


## Sources
* https://www.geeksforgeeks.org/python/how-to-use-yfinance-api-with-python/
* https://ranaroussi.github.io/yfinance/